In [1]:
import pandas as pd
from fredapi import Fred
import os
from openpyxl import load_workbook
from typing import List, Tuple, Union
from pathlib import Path

In [2]:
api="958ccd9c67808caf9f941367daf6e812"
fred = Fred(api_key=api)

In [20]:
monthly_tickers = [
    "RECPROUSM156N",
    "PCEDGC96",
    'PCENDC96',
    "PCESC96",
    "RSAFS",
    "DSPIC96",
    "PSAVERT",
    "TOTALSA",
    "REVOLSL",
    "NONREVSL",
    "DGORDER",
    'ADXDNO',
    "INDPRO",
    "TCU",
    "HOUST",
    "PERMIT",
    "HSN1F",
    "EXHOSLUSM495S",
    "PAYEMS",
    "ADPMNUSNERSA",
    "UNRATE",
    "U6RATE",
    "CIVPART",
    "EMRATIO",
    "JTSJOR",
    "JTSHIR",
    "JTSTSR",
    "AWHAETP",
    "CPIAUCSL",
    "CPILFESL",
    "PPIFIS",
    "PCEPI",
    "PCEPILFE",
    "UMCSENT",
    'CES0500000003',
    "AHETPI",
    "CSUSHPINSA",
    "IQ",
    "IR",
    'BOPTEXP',
    'BOPTIMP',
    'BOPSTB',
]

quarterly_tickers = [
    "GDPC1",
    "NGDPSAXDCUSQ",
    "GDPNOW",
    "PCECC96",
    "PNFIC1",
    "PRFIC1",
    "GCE",
    "ECIWAG",
    'PRS85006092'
]

weekly_tickers = [
    "ICSA",
    "CCSA",
    "MORTGAGE30US"
]

daily_tickers = [
    "T5YIFR",
    "T10YIE",
    "DTWEXBGS",
    "DFF",
    "DGS2",
    "DGS5",
    "DGS10",
    "DBAA"
]

In [44]:
def Fetch(ticker_list, fred, freq,warning_list=[]):
    values_dfs = []
    dates_dfs = []
    # Set Frequency
    f = str(freq).strip().upper()
    if f in ("MONTHLY", "M"):
        period_code = "M"
    elif f in ("QUARTERLY", "Q"):
        period_code = "Q"
    elif f in ("WEEKLY", "W"):
        period_code = "W"
    elif f in ("ANNUAL", "A", "Y"):
        period_code = "A"
    elif f in ("DAILY", "D"):
        period_code = "D"
    else:
        raise ValueError(f"Unsupported frequency: {freq}")

    # Get Data through Fred. throw an error if unable to fetch
    for ticker in [t for t in ticker_list if t not in warning_list]:
        try:
            s = fred.get_series_all_releases(ticker)
        except Exception as e:
            print(f"Warning: couldn't fetch {ticker}: {e}")
            continue
        s["date"] = pd.to_datetime(s["date"])
        s["realtime_start"] = pd.to_datetime(s["realtime_start"])

        # Normalize timestamps by converting each release_date to the start of its time period
        # (e.g., start of day, month, quarter, etc.) for consistent grouping and comparison
        if period_code == "D":
            s["Time"] = s["date"].dt.floor("D")
        else:
            s["Time"] = s["date"].dt.to_period(period_code).dt.to_timestamp()

        # Group by the period and take the last (i.e., the latest release for that period)
        # ticker is the value column and real_time_start is the release date column
        grouped = s.groupby("Time").agg({"value": "last", "realtime_start": "last"})
        values_dfs.append(grouped[["value"]].rename(columns={"value": ticker}))
        dates_dfs.append(grouped[["realtime_start"]].rename(columns={"realtime_start": ticker}))

    # deal with the variables in
    for ticker in warning_list:
        try:
            s = fred.get_series(ticker)
        except Exception as e:
            print(f"Warning: couldn't fetch {ticker}: {e}")
            continue

        # Change to datetime
        s.index = pd.to_datetime(s.index)
        df = s.reset_index()
        df.columns = ["release_date", ticker]

        # Normalize timestamps by converting each release_date to the start of its time period
        # (e.g., start of day, month, quarter, etc.) for consistent grouping and comparison
        # here because release data is temporarily not available because the series is not available in fred. I just keep the release date blank
        if period_code == "D":
            df["Time"] = df["release_date"].dt.floor("D")
        else:
            df["Time"] = df["release_date"].dt.to_period(period_code).dt.to_timestamp()

        grouped = df.groupby("Time").agg({ticker: "last", "release_date": "last"})
        values_dfs.append(grouped[[ticker]])
        dates_dfs.append(grouped[["release_date"]].rename(columns={"release_date": ticker}))

    if not values_dfs:
        return pd.DataFrame(), pd.DataFrame()

    # concat the series of each economic data into a table
    values = pd.concat(values_dfs, axis=1, sort=True)
    dates = pd.concat(dates_dfs, axis=1, sort=True)
    values.index.name = "Time"
    dates.index.name = "Time"

    return values, dates

In [6]:
# to csv
# find project root relative to this script
# Save the data table to the folder "Raw Data"
base_dir = Path.cwd().parent.parent
print(base_dir)
raw_location = base_dir / "Raw Data"
raw_location.mkdir(parents=True, exist_ok=True)

C:\Users\hp\Desktop\Duke Research Opportunity\Prof. Aguilar Macro Dashboard\MacroDashBoard


In [45]:
monthly_values, monthly_dates = Fetch(monthly_tickers, fred, "M",['EXHOSLUSM495S'])
quarterly_values, quarterly_dates = Fetch(quarterly_tickers, fred, "Q")
weekly_values, weekly_dates = Fetch(weekly_tickers, fred, "W")
daily_values, daily_dates = Fetch(daily_tickers, fred, "D",daily_tickers)

In [47]:
daily_values

,T5YIFR,T10YIE,DTWEXBGS,DFF,DGS2,DGS5,DGS10,DBAA
Time,,,,,,,,
1954-07-01,NaN,NaN,NaN,1.13,NaN,NaN,NaN,NaN
1954-07-02,NaN,NaN,NaN,1.25,NaN,NaN,NaN,NaN
1954-07-03,NaN,NaN,NaN,1.25,NaN,NaN,NaN,NaN
1954-07-04,NaN,NaN,NaN,1.25,NaN,NaN,NaN,NaN
1954-07-05,NaN,NaN,NaN,0.88,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2025-11-01,NaN,NaN,NaN,3.86,NaN,NaN,NaN,NaN
2025-11-02,NaN,NaN,NaN,3.86,NaN,NaN,NaN,NaN
2025-11-03,2.21,2.31,NaN,3.87,3.60,3.72,4.13,5.84


In [13]:
monthly_dates

,realtime_start,date,value
0,1992-12-22 00:00:00,1946-01-01 00:00:00,199.7
1,1996-01-19 00:00:00,1946-01-01 00:00:00,NaT
2,1997-05-07 00:00:00,1946-01-01 00:00:00,210.4
3,1999-10-28 00:00:00,1946-01-01 00:00:00,NaT
4,1992-12-22 00:00:00,1946-04-01 00:00:00,207.7
...,...,...,...
3226,2025-06-26 00:00:00,2025-01-01 00:00:00,29962.047
3227,2025-09-25 00:00:00,2025-01-01 00:00:00,30042.113
3228,2025-07-30 00:00:00,2025-04-01 00:00:00,30331.117
3229,2025-08-28 00:00:00,2025-04-01 00:00:00,30353.902


In [ ]:
def aggregate_by_representation_period(df, ticker, period_code="M"):

    # Ensure datetimes
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])
    df["real_time_start"] = pd.to_datetime(df["real_time_start"])

    # Derive grouping index "Time" from the representation-period column.
    # If period_code == "D" use the day; otherwise convert date->period->timestamp start
    if period_code == "D":
        df["Time"] = df["date"].dt.floor("D")
    else:
        df["Time"] = df["date"].dt.to_period(period_code).dt.to_timestamp()

    # Sort by actual release time so the last entry per Time is the latest release
    df = df.sort_values("real_time_start")

    # Group by the period and take the last (i.e., the latest release for that period)
    grouped = df.groupby("Time").agg({ticker: "last", "real_time_start": "last"})

    # values: latest value per represented period
    values = grouped[[ticker]].copy()
    # dates: latest release time per represented period (column name aligned to ticker)
    dates = grouped[["real_time_start"]].rename(columns={"real_time_start": ticker}).copy()

    values.index.name = "Time"
    dates.index.name = "Time"

    return values, dates

In [56]:
# GetData from the csv files
def NormalizeFreq(code: str) -> str:
    """Return canonical single-letter code: 'A','Q','M','W','D' or raise."""
    fm = {
        ("MONTHLY", "M"): "M",
        ("QUARTERLY", "Q"): "Q",
        ("WEEKLY", "W"): "W",
        ("ANNUAL", "A", "Y"): "A",
        ("DAILY", "D"): "D",
    }
    code_up = code.upper()
    for keys, val in fm.items():
        if code_up in keys:
            return val
    raise ValueError(f"Unsupported frequency: {code}")

def GetData(ticker: str, freq: str, n_lags: int = 4):
    period_code=NormalizeFreq(freq)
    base = base_dir / "Raw Data"
    dates_path = os.path.join(base, f"{period_code}_Dates.csv")
    values_path = os.path.join(base, f"{period_code}_Values.csv")

    dates = pd.read_csv(dates_path,index_col=0, parse_dates=True)
    values = pd.read_csv(values_path,index_col=0, parse_dates=True)

    if ticker not in values.columns:
        raise KeyError(f"Ticker '{ticker}' not found in values file.")

    last_idx = values[ticker].last_valid_index()
    last_pos = values.index.get_loc(last_idx)
    current_period = last_idx
    latest_update = dates[ticker].iloc[last_pos]
    value_col = values[ticker]
    return latest_update, value_col, current_period

In [52]:
current_period,latext_update,value_col=GetData("PCEDGC96", freq="M")

In [55]:
current_period

Timestamp('2025-08-01 00:00:00')

In [57]:
def GetLevel(ticker: str, freq: str, n_lags: int = 4):
    latest_date, value_col, current_period = GetData(ticker, freq, n_lags)
    last_idx = value_col.last_valid_index()
    last_pos = value_col.index.get_loc(last_idx)
    start_pos = max(0, last_pos - n_lags)
    latest_values = value_col.iloc[start_pos: last_pos + 1].tolist()
    return latest_date, latest_values, current_period

def GetDelta(ticker: str, freq: str, agg_freq: str, n_lags: int = 4, pct: bool = True) -> Tuple[str, pd.Series, pd.Timestamp]:
    """
    Return (latest_date, delta_series, current_period) where delta_series is the same index as value_col
    containing differences (level diffs) or percent changes depending on `pct`.
    - `freq` is the native frequency of the data stored (e.g., "M" or "MONTHLY").
    - `agg_freq` is the aggregation period for the delta (e.g., "A" for year-over-year).
    - If value_col has a DatetimeIndex, the function prefers date-aware shifts (DateOffset).
    - If value_col has a plain integer index, the function uses integer shifts where we can derive them.
    """
    _INT_SHIFT_TABLE = {
        ("M", "A"): 12,
        ("M", "Q"): 3,
        ("Q", "A"): 4,
        ("Q", "M"): None,
        ("W", "M"): None,
        ("D", "M"): None,
        ("M", "M"): 1,
        ("Q", "Q"): 1,
        ("A", "A"): 1,
        ("W", "W"): 1,
        ("D", "D"): 1,
    }

    # normalize freq inputs
    freq_code = NormalizeFreq(freq)
    agg_code = NormalizeFreq(agg_freq)

    # Get Data (now returns latest_date, value_col, current_period)
    latest_date, value_col, current_period = GetData(ticker, freq_code, n_lags)
    last_idx = value_col.last_valid_index()
    last_pos = value_col.index.get_loc(last_idx)
    start_pos = max(0, last_pos - n_lags)

    # Same-frequency change (e.g., MoM when agg == freq)
    if agg_code == freq_code:
        shift_arg = 1
        if pct:
            result = value_col.pct_change(shift_arg, fill_method=None)
        else:
            result = value_col.diff(shift_arg)
        result = result.iloc[start_pos : last_pos + 1].tolist()
        return latest_date, result, current_period

    # Default: try datetime-aware shifts first
    idx = value_col.index
    if isinstance(idx, pd.DatetimeIndex) or pd.api.types.is_datetime64_any_dtype(idx):

        if freq_code == "M" and agg_code == "A":
            offset = pd.DateOffset(months=12)
        elif freq_code == "M" and agg_code == "Q":
            offset = pd.DateOffset(months=3)
        elif freq_code == "Q" and agg_code == "A":
            offset = pd.DateOffset(months=12)
        elif freq_code == "Q" and agg_code == "M":
            raise ValueError("Unsupported conversion: quarter-index to month-based aggregation when using DatetimeIndex.")
        elif freq_code == "W" and agg_code == "A":
            offset = pd.DateOffset(weeks=52)
        elif freq_code == "D" and agg_code == "A":
            offset = pd.DateOffset(years=1)
        else:
            raise ValueError(f"Unsupported date-based conversion: {freq_code} -> {agg_code}")

        # Move historical values so prev[t] == value_col[t - offset]
        prev = value_col.shift(freq=offset)
        prev = prev.reindex(value_col.index)

        if pct:
            result = (value_col - prev) / prev
        else:
            result = value_col - prev

        result = result.iloc[start_pos : last_pos + 1].tolist()
        return latest_date, result, current_period

    # If not a datetime index, fall back to integer shifts
    key = (freq_code, agg_code)
    shift = _INT_SHIFT_TABLE.get(key, None)
    if shift is None:
        if freq_code == "Q" and agg_code == "A":
            shift = 4
        elif freq_code == "M" and agg_code == "A":
            shift = 12
        elif freq_code == "M" and agg_code == "Q":
            shift = 3
        else:
            raise ValueError(f"Unsupported conversion for integer-indexed series: {freq_code} -> {agg_code}")

    shift = int(shift)
    if pct:
        result = value_col.pct_change(shift, fill_method=None)
    else:
        result = value_col.diff(shift)

    result = result.iloc[start_pos : last_pos + 1].tolist()
    return latest_date, result, current_period


In [6]:
base_dir = Path.cwd().parent.parent

In [58]:
raw_location = base_dir / "Raw Data"
raw_location.mkdir(parents=True, exist_ok=True)
daily_values=pd.read_csv(raw_location/"D_Values.csv")
monthly_values=pd.read_csv(raw_location/"M_Values.csv")
monthly_dates=pd.read_csv(raw_location/"M_Dates.csv")

In [59]:
def AddRealAvgEarning():
    monthly_values['RCES0500000003']=monthly_values['CES0500000003']/(monthly_values['PCEPI']/100)
    monthly_dates['RCES0500000003']=monthly_dates['CES0500000003']

AddRealAvgEarning()
monthly_values

,Time,RECPROUSM156N,PCEDGC96,PCENDC96,PCESC96,RSAFS,DSPIC96,PSAVERT,TOTALSA,REVOLSL,...,AHETPI,CSUSHPINSA,IQ,IR,BOPTEXP,BOPTIMP,BOPSTB,EXHOSLUSM495S,RCES0500000003*,RCES0500000003
0,1919-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1919-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1919-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1919-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1919-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1277,2025-06-01,0.34,2113.9,3522.8,10863.2,723033.0,18036.2,5.0,16.042,1300321.06,...,31.26,331.685,152.4,140.7,279650.0,338736.0,26631.0,3930000.0,28.648525,28.648525
1278,2025-07-01,0.64,2152.2,3528.1,10888.2,727414.0,18077.3,4.8,16.896,1311491.90,...,31.34,331.003,152.8,141.0,280464.0,358775.0,25565.0,4010000.0,28.696563,28.696563
1279,2025-08-01,0.96,2171.4,3547.1,10910.9,732010.0,18097.2,4.6,16.802,1305533.90,...,31.46,330.022,153.2,141.4,NaN,NaN,NaN,4000000.0,28.699375,28.699375
1280,2025-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4060000.0,NaN,NaN


In [54]:
monthly_values.shift(freq=pd.DateOffset(months=12))

,SAHMREALTIME,RECPROUSM156N,PCEDGC96,PCENDC96,PCESC96,RSAFS,DSPIC96,PSAVERT,TOTALSA,REVOLSL,...,T5YIFR,CES0500000003,AHETPI,CSUSHPINSA,DTWEXBGS,IQ,IR,BOPTEXP,BOPTIMP,BOPSTB
Time,,,,,,,,,,,,,,,,,,,,,
1920-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1920-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1920-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1920-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1920-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-06-01,0.17,0.34,2113.9,3522.8,10863.2,723033.0,18036.2,5.0,16.042,1300321.06,...,2.27,36.31,31.26,331.671,119.8269,152.4,140.7,279650.0,338736.0,26631.0
2026-07-01,0.10,0.64,2152.2,3528.1,10888.2,727414.0,18077.3,4.8,16.896,1311491.90,...,2.32,36.43,31.34,331.127,122.1088,152.8,141.0,280464.0,358775.0,25565.0
2026-08-01,0.13,0.96,2171.4,3547.1,10910.9,732010.0,18097.2,4.6,16.802,1305533.90,...,2.35,36.53,31.46,NaN,120.6028,153.2,141.4,NaN,NaN,NaN


In [32]:
# --- Constants ---
folder = base_dir / "MacroDashboard Versions"
input_xlsx = folder / "Economic Dashboard V1-Template.xlsx"
output_xlsx = folder / "Economic Dashboard V1-Template-UPDATE.xlsx"

wb = load_workbook(input_xlsx)
ws = wb.active
start_row = 3
max_row = ws.max_row
def _to_python_datetime(dt: Union[pd.Timestamp, str, None]):
    """
    Convert pandas.Timestamp -> python datetime for openpyxl.
    If dt is None or unparseable, return it unchanged.
    """
    if isinstance(dt, pd.Timestamp):
        return dt.to_pydatetime()
    return dt

def write_panel_for_row(
    row: int,
    ticker_col: str,
    freq_col: str,
    date_col: str,
    units_col: str,
    present_col: str,
    lag_cols: List[str],
    n_lags: int = 4,
) -> None:
    """
    Write one row for a panel (left or right).
    - ticker_col, freq_col, date_col, present_col: column letters (e.g. "B", "E", "C", "F")
    - lag_cols: list of lag column letters in order [Lag1_col, Lag2_col, ...]
    - n_lags: number of lags expected (defaults to 4)
    """
    # read ticker and freq values from the worksheet
    raw_ticker = ws[f"{ticker_col}{row}"].value
    raw_freq = ws[f"{freq_col}{row}"].value
    raw_units = ws[f"{units_col}{row}"].value

    # If ticker cell is empty -> skip row
    if raw_ticker is None or str(raw_ticker).strip() == "" or str(raw_freq).strip() == "Freq":
        return

    # normalize strings safely
    ticker = str(raw_ticker).strip()
    freq = str(raw_freq).strip() if raw_freq is not None else "M"
    units = str(raw_units).strip().lower()

    # get data using your GetLevel function (assumed defined/imported)
    if('delta' not in units):
        try:
            latest_date, recent_values = GetLevel(ticker, freq, n_lags=n_lags)
        except Exception as exc:
            # write error to date cell and skip writing values for this row
            ws[f"{date_col}{row}"].value = f"ERR: {exc}"
            return
    else:
        pct="%" in units
        agg=units[units.index("/")+1:units.index("/")+2]
        try:
            latest_date, recent_values = GetDelta(ticker, freq, agg, n_lags=n_lags, pct=pct)
        except Exception as exc:
             # write error to date cell and skip writing values for this row
            ws[f"{date_col}{row}"].value = f"ERR: {exc}"
            return

    # recent_values expected chronological oldest ... most recent
    values = list(recent_values)

    # Ensure fixed length = n_lags + 1 by padding at front (older side) with None
    expected_len = n_lags + 1
    if len(values) < expected_len:
        pad_len = expected_len - len(values)
        values = [None] * pad_len + values

    # Present is last element, lag1 is second-last, etc.
    present_value = values[-1]
    lag_values = []
    for i in range(1, len(lag_cols) + 1):
        # i=1 -> lag1 => values[-2], general: values[-1 - i]
        idx = -1 - i
        # safe access: if index out of range, return None
        try:
            lag_values.append(values[idx])
        except IndexError:
            lag_values.append(None)

    # write date (convert pandas Timestamp -> python datetime for openpyxl)
    ws[f"{date_col}{row}"].value = _to_python_datetime(latest_date)

    # write present and lag values
    ws[f"{present_col}{row}"].value = present_value
    for col_letter, lag_value in zip(lag_cols, lag_values):
        ws[f"{col_letter}{row}"].value = lag_value


# Output section is the left panel, and other three sections are in the right panel.
# Can automate the identification of columns for each panel
# Left panel: B = ticker, E = freq, C = Latest Date, F = Present, G,H,I,J = Lag1..Lag4
left_panel = {
    "ticker_col": "B",
    "freq_col": "E",
    "date_col": "C",
    "units_col": "D",
    "present_col": "F",
    "lag_cols": ["G", "H", "I", "J"],
}

# Right panel: M = ticker, P = freq, N = Latest Date, Q = Present, R,S,T,U = Lag1..Lag4
right_panel = {
    "ticker_col": "M",
    "freq_col": "P",
    "date_col": "N",
    "units_col": "O",
    "present_col": "Q",
    "lag_cols": ["R", "S", "T", "U"],
}

# iterate and write
for r in range(start_row, max_row + 1):
    write_panel_for_row(r, **left_panel)
    write_panel_for_row(r, **right_panel)

wb.save(output_xlsx)
